In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import mdtraj
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *

In [2]:
topoFile = r'D:\MD\ASAP-sim\ASAP3b\A-GFP\system_dabbled.psf'
trajFile = r'D:\MD\ASAP-sim\ASAP3b\A-GFP\run_1.nc'
traj = mdtraj.load(trajFile, top=topoFile)
topo = traj.topology

- residues of interest
    - F413 PHE, 365 in PyMol
    - Q414 GLN, 366 in PyMol

In [3]:
def getResAtoms(topo, seq):
    atoms = topo.select("protein and resSeq %d" % seq)
    res = topo.atom(atoms[0]).residue
    print('\n', res)
    return atoms

print(getResAtoms(topo, 365))
print(getResAtoms(topo, 366))


 PHE365
[92386 92387 92388 92389 92390 92391 92392 92393 92394 92395 92396 92397
 92398 92399 92400 92401 92402 92403 92404 92405]

 GLN366
[92406 92407 92408 92409 92410 92411 92412 92413 92414 92415 92416 92417
 92418 92419 92420 92421 92422]


In [4]:
class EnergyFinder:
    def __init__(self, traj):
        forceField = ForceField('charmm36.xml', 'charmm36/water.xml')
        topology = traj.topology.to_openmm()
        print(topology)
        system = forceField.createSystem(topology)
        integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)
        simulation = Simulation(topology, system, integrator)
        self.sim = simulation
        self.traj = traj
        
    def getStep(self, step):
        self.sim.context.setPositions(self.traj.xyz[step])
        state = self.sim.context.getState(getEnergy=True, getForces=True)
        potentialEnergy = state.getPotentialEnergy()
        kineticEnergy = state.getKineticEnergy()
        force = state.getForces(asNumpy=True)
        
energyFinder = EnergyFinder(traj)
energyFinder.getStep(1)

<Topology; 5 chains, 20533 residues, 92627 atoms, 92376 bonds>


ValueError: No template found for residue 108 (HOH).  The set of atoms matches TIP3, but the bonds are different.  Perhaps the chain is missing a terminal group?